# Enrichissement Hybride LLM - Paris.fr Appels à Projets

Ce notebook scrape ET enrichit les données avec Claude Sonnet 4.5 pour extraire:
- **Scraping complet** du site Paris.fr
- **Résumés structurés** via LLM
- **Montants (min/max)** extraits automatiquement
- **Catégories et tags** intelligents
- **Public cible** identifié
- **Modalités et démarches** structurées
- **Extraction PDF** des règlements

**Approche:** Workflow complet et indépendant = Scraping + LLM Claude pour enrichissement

## 1. Imports et configuration

In [1]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
from datetime import datetime
import json
from urllib.parse import urljoin
import time
import os
from dotenv import load_dotenv
import tempfile
import hashlib
import itertools

# Imports LLM
from anthropic import Anthropic
import pypdf

In [2]:
# Charger les variables d'environnement
load_dotenv(override=True)

# Vérifier Claude API key
claude_api_key = os.getenv('ANTHROPIC_API_KEY')
if claude_api_key:
    print(f"✅ ANTHROPIC_API_KEY trouvée: {claude_api_key[:10]}...")
else:
    print(f"❌ ANTHROPIC_API_KEY non trouvée dans .env")
    print(f"   ⚠️ Vous devez ajouter: ANTHROPIC_API_KEY=sk-ant-xxxxxx")

✅ ANTHROPIC_API_KEY trouvée: sk-ant-api...


## 2. Configuration scraper

In [3]:
# Configuration du scraper
BASE_URL = "https://www.paris.fr/appels-a-projets"
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
}

print(f"✅ Configuration prête")
print(f"   Base URL: {BASE_URL}")
print(f"   Dépendances: requests, BeautifulSoup4, pandas, anthropic, pypdf")

✅ Configuration prête
   Base URL: https://www.paris.fr/appels-a-projets
   Dépendances: requests, BeautifulSoup4, pandas, anthropic, pypdf


## 3. Scraper les appels à projets de Paris.fr

In [4]:
# Récupérer le HTML de la page
print(f"🔄 Fetching {BASE_URL}...")
try:
    response = requests.get(BASE_URL, headers=HEADERS, timeout=10)
    response.raise_for_status()
    html_content = response.text
    print(f"✅ Page récupérée ({len(html_content)} caractères)")
except Exception as e:
    print(f"❌ Erreur: {str(e)}")
    html_content = None

🔄 Fetching https://www.paris.fr/appels-a-projets...
✅ Page récupérée (312287 caractères)


In [5]:
# Parser le HTML
if html_content:
    soup = BeautifulSoup(html_content, 'html.parser')
    print(f"✅ HTML parsé avec BeautifulSoup")
    print(f"   Titre: {soup.title.string if soup.title else 'N/A'}")
else:
    soup = None
    print("⚠️ Pas de contenu HTML")

✅ HTML parsé avec BeautifulSoup
   Titre: Appels à projets - Ville de Paris


In [6]:
def scrape_paris_aap(soup):
    """Scrape les AAP du site Paris.fr"""
    aap_list = []
    
    if not soup:
        return aap_list
    
    # Chercher les conteneurs des AAP
    article_containers = soup.find_all(['article', 'div'], class_=re.compile(r'(appel|project|call|aap)', re.I))
    
    if not article_containers:
        article_containers = soup.find_all('a', href=re.compile(r'appel|project', re.I))
    
    print(f"📝 Trouvé {len(article_containers)} conteneurs potentiels")
    
    for container in article_containers:
        try:
            aap = {}
            
            # Titre
            title_elem = container.find(['h2', 'h3', 'h4', 'a'])
            aap['titre'] = title_elem.get_text(strip=True) if title_elem else 'N/A'
            
            # URL
            if container.name == 'a':
                aap['url_source'] = urljoin(BASE_URL, container.get('href', ''))
            else:
                link = container.find('a', href=True)
                aap['url_source'] = urljoin(BASE_URL, link.get('href', '')) if link else None
            
            # Description/résumé
            desc = container.find(['p', 'span'], class_=re.compile(r'(desc|summary|excerpt)', re.I))
            aap['resume'] = desc.get_text(strip=True) if desc else None
            
            # Dates
            text_content = container.get_text(' ')
            dates = re.findall(r'\d{1,2}/\d{1,2}/\d{4}', text_content)
            if len(dates) >= 2:
                try:
                    aap['date_publication'] = pd.to_datetime(dates[0], format='%d/%m/%Y').date()
                    aap['date_limite'] = pd.to_datetime(dates[1], format='%d/%m/%Y').date()
                except:
                    aap['date_publication'] = None
                    aap['date_limite'] = None
            elif len(dates) == 1:
                try:
                    aap['date_limite'] = pd.to_datetime(dates[0], format='%d/%m/%Y').date()
                    aap['date_publication'] = None
                except:
                    aap['date_publication'] = None
                    aap['date_limite'] = None
            else:
                aap['date_publication'] = None
                aap['date_limite'] = None
            
            # Montant
            amounts = re.findall(r'(\d+[\s.,]*\d*)\s*(?:€|euros?|EUROS?)', text_content, re.I)
            if amounts:
                try:
                    cleaned = amounts[-1].replace(' ', '').replace('.', '').replace(',', '.')
                    aap['montant_max'] = float(cleaned)
                except:
                    aap['montant_max'] = None
            else:
                aap['montant_max'] = None
            
            # Organisme
            org = container.find(['span', 'p'], class_=re.compile(r'(organisme|provider|publisher)', re.I))
            aap['organisme'] = org.get_text(strip=True) if org else 'Mairie de Paris'
            
            # Catégories
            categories_elem = container.find_all(['span', 'a'], class_=re.compile(r'(categor|tag|theme)', re.I))
            if categories_elem:
                aap['categories'] = [cat.get_text(strip=True) for cat in categories_elem]
            else:
                aap['categories'] = None
            
            # ID unique
            aap['id_record'] = f"paris_{datetime.now().strftime('%Y%m%d%H%M%S')}_{hash(aap['titre']) % 10000}"
            
            aap_list.append(aap)
        except Exception as e:
            print(f"  ⚠️ Erreur parsing: {str(e)}")
            continue
    
    return aap_list

# Scraper
aap_data = scrape_paris_aap(soup)
print(f"\n✅ {len(aap_data)} appels à projets extraits")

📝 Trouvé 12 conteneurs potentiels

✅ 12 appels à projets extraits


## 4. Créer et nettoyer le DataFrame

In [7]:
# Créer DataFrame
mapped_df_paris = pd.DataFrame(aap_data)
print(f"📊 DataFrame créé: {mapped_df_paris.shape}")
print(f"   Colonnes: {list(mapped_df_paris.columns)}")

📊 DataFrame créé: (12, 9)
   Colonnes: ['titre', 'url_source', 'resume', 'date_publication', 'date_limite', 'montant_max', 'organisme', 'categories', 'id_record']


In [8]:
# Fonction de nettoyage du texte
def clean_text(text):
    if not isinstance(text, str):
        return text
    text = re.sub(r'<[^>]+>', '', text)
    html_entities = {
        '&eacute;': 'é', '&icirc;': 'î', '&agà;': 'à',
        '&nbsp;': ' ', '&quot;': '\"', '&amp;': '&',
        '&rsquo;': "'", '&ldquo;': '\u201c', '&rdquo;': '\u201d'
    }
    for entity, char in html_entities.items():
        text = text.replace(entity, char)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# Appliquer le nettoyage
for col in mapped_df_paris.select_dtypes(include=['object']).columns:
    if col not in ['categories']:
        mapped_df_paris[col] = mapped_df_paris[col].apply(clean_text)

print("✅ Texte nettoyé")

✅ Texte nettoyé


In [9]:
# Ajouter colonnes manquantes
colonnes_requises = ['public_cible', 'taux_financement', 'contact', 'modalite', 'demarches', 'mots_cles', 'objectif', 'montant_min', 'note', 'tags']

for col in colonnes_requises:
    if col not in mapped_df_paris.columns:
        mapped_df_paris[col] = None

if 'perimetre_geo' not in mapped_df_paris.columns:
    mapped_df_paris['perimetre_geo'] = 'Paris'

if 'fingerprint' not in mapped_df_paris.columns:
    def create_fingerprint(row):
        titre = str(row['titre']) if pd.notna(row['titre']) else ''
        organisme = str(row['organisme']) if pd.notna(row['organisme']) else ''
        date_limite = str(row['date_limite']) if pd.notna(row['date_limite']) else ''
        combined = f"{titre}|{organisme}|{date_limite}"
        return hashlib.md5(combined.encode()).hexdigest()[:12]
    
    mapped_df_paris['fingerprint'] = mapped_df_paris.apply(create_fingerprint, axis=1)

print(f"✅ DataFrame préparé: {mapped_df_paris.shape}")
print(f"   Colonnes finales: {list(mapped_df_paris.columns)}")

✅ DataFrame préparé: (12, 21)
   Colonnes finales: ['titre', 'url_source', 'resume', 'date_publication', 'date_limite', 'montant_max', 'organisme', 'categories', 'id_record', 'public_cible', 'taux_financement', 'contact', 'modalite', 'demarches', 'mots_cles', 'objectif', 'montant_min', 'note', 'tags', 'perimetre_geo', 'fingerprint']


## 5. Fonctions pour extraction PDF

In [10]:
def extract_pdf_text(pdf_url, max_pages=3):
    """Extraire le texte d'un PDF depuis une URL"""
    try:
        response = requests.get(pdf_url, timeout=10)
        response.raise_for_status()
        
        with tempfile.NamedTemporaryFile(suffix='.pdf', delete=False) as tmp:
            tmp.write(response.content)
            tmp_path = tmp.name
        
        reader = pypdf.PdfReader(tmp_path)
        text = ''
        for page_num, page in enumerate(reader.pages[:max_pages]):
            text += page.extract_text() + '\n'
        
        os.remove(tmp_path)
        return text if text.strip() else None
    except Exception as e:
        print(f"  ⚠️ Erreur PDF {pdf_url}: {str(e)[:50]}")
        return None


def find_pdf_links(soup, base_url):
    """Trouver les liens PDF dans une page"""
    pdf_links = []
    for link in soup.find_all('a', href=True):
        href = link.get('href', '')
        text = link.get_text().lower()
        
        if ('pdf' in href.lower() or 
            any(keyword in text for keyword in ['reglement', 'document', 'cahier', 'guide'])):
            full_url = urljoin(base_url, href)
            if full_url not in pdf_links:
                pdf_links.append(full_url)
    
    return pdf_links[:2]

print("✅ Fonctions PDF créées")

✅ Fonctions PDF créées


## 6. Classe LLMEnricher

In [12]:
class LLMEnricher:
    """Enrichir les données AAP avec Claude Sonnet 4.5"""
    
    def __init__(self, api_key=None, model='claude-sonnet-4-5'):
        self.api_key = api_key or os.getenv('ANTHROPIC_API_KEY')
        self.model = model
        self.client = Anthropic(api_key=self.api_key) if self.api_key else None
        self.max_retries = 3
        self.retry_delay = 1
        
        if not self.client:
            raise ValueError('❌ ANTHROPIC_API_KEY non trouvée')
    
    def extract_full_page(self, url, html_content, pdf_texts=None):
        """Extraire toutes les données manquantes d'une page avec retry logic"""
        
        if not self.client:
            return None
        
        soup = BeautifulSoup(html_content, 'html.parser')
        text_content = soup.get_text('\n')
        
        if pdf_texts:
            text_content += '\n\n--- DOCUMENTS PDF ---\n'
            text_content += '\n\n'.join(pdf_texts)
        
        text_content = text_content[:16000]
        
        prompt = f"""Tu es un expert en analyse d'appels à projets parisiens.
        
Analyse cette page et extrais les informations manquantes en JSON valide:

{{
   "resume": "Résumé en 1-2 phrases (max 300 caractères)",
  "montant_max": montant maximum en euros (nombre ou null),
  "montant_min": montant minimum en euros (nombre ou null),
  "taux_financement": "pourcentage ou description (null si non trouvé)",
  "categories": ["liste", "de", "catégories"],
  "public_cible": ["associations", "PME", "collectivités"],
  "mots_cles": ["mots-clés", "pertinents"],
  "objectif": "Quel est l'objectif principal",
  "modalite": "Conditions principales",
  "demarches": "Comment candidater",
  "contact": "Email ou téléphone si trouvé (ou null)"
}}

IMPORTANT:
- Retourne UNIQUEMENT du JSON valide
- Si une info n'existe pas, mets null
- Les montants doivent être des nombres
- Les listes doivent être des arrays JSON

Contenu:
{text_content}"""
        
        for attempt in range(self.max_retries):
            try:
                message = self.client.messages.create(
                    model=self.model,
                    max_tokens=1024,
                    messages=[{"role": "user", "content": prompt}]
                )
                
                response_text = message.content[0].text
                response_text = response_text.replace('```json', '').replace('```', '')
                
                return json.loads(response_text.strip())
            except json.JSONDecodeError as e:
                print(f"    ❌ JSON parsing error: {str(e)[:50]}")
                return None
            except Exception as e:
                error_msg = str(e)
                print(f"    ❌ Attempt {attempt + 1}/{self.max_retries} - Error: {error_msg[:80]}")
                if attempt < self.max_retries - 1:
                    wait_time = self.retry_delay * (2 ** attempt)
                    print(f"    ⏳ Retrying in {wait_time}s...")
                    time.sleep(wait_time)
                else:
                    print(f"    ❌ Max retries exceeded")
                    return None

print("✅ Classe LLMEnricher créée")

✅ Classe LLMEnricher créée


## 7.5 Aperçu des PDFs extraits

In [13]:
# Afficher un aperçu des PDFs avant le LLM
if mapped_df_paris is not None and len(mapped_df_paris) > 0:
    print("📄 Aperçu de l'extraction PDF:")
    print("="*80)
    
    pdf_preview_data = []
    
    for idx, row in itertools.islice(mapped_df_paris.iterrows(), 3):  # Limiter à 3 aperçus
        url = row.get('url_source')
        titre = str(row.get('titre', 'N/A'))[:50]
        
        if not url or pd.isna(url):
            continue
        
        try:
            response = requests.get(url, headers=HEADERS, timeout=10)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.text, 'html.parser')
            pdf_links = find_pdf_links(soup, url)
            
            if pdf_links:
                print(f"\n📌 {titre}")
                print(f"   URL: {url[:60]}...")
                print(f"   PDFs trouvés: {len(pdf_links)}")
                
                for pdf_idx, pdf_url in enumerate(pdf_links, 1):
                    pdf_filename = pdf_url.split('/')[-1][:50]
                    print(f"   \n   {pdf_idx}️⃣ {pdf_filename}")
                    print(f"      URL: {pdf_url[:70]}...")
                    
                    pdf_text = extract_pdf_text(pdf_url, max_pages=1)
                    if pdf_text:
                        text_preview = pdf_text[:200].replace('\n', ' ').strip()
                        print(f"      Contenu (1ère page): {text_preview}...")
                    else:
                        print(f"      ❌ Impossible d'extraire le texte")
        except Exception as e:
            print(f"\n⚠️  Erreur pour {titre}: {str(e)[:50]}")
        
        time.sleep(0.5)
    
    print("\n" + "="*80)
    print("✅ Aperçu terminé - Prêt pour l'enrichissement LLM")

📄 Aperçu de l'extraction PDF:

📌 Appel à projets pour le partage de la culture scie
   URL: https://www.paris.fr/pages/appel-a-projets-2024-pour-le-part...
   PDFs trouvés: 2
   
   1️⃣ budget-type-aapcs2026-v4Hz.pdf
      URL: https://cdn.paris.fr/paris/2025/12/23/budget-type-aapcs2026-v4Hz.pdf...
      Contenu (1ère page): Document type à télécharger, à compléter,   puis à joindre au dossier de candidature    Budget prévisionnel du projet      CHARGES MONTANT PRODUITS MONTANT  Achats   Recette des activités   Prestation...
   
   2️⃣ aap_culture-scientifique_2026-DqVw.pdf
      URL: https://cdn.paris.fr/paris/2025/12/23/aap_culture-scientifique_2026-Dq...
      Contenu (1ère page): o  o  o...

📌 Appel à projets « Prévenir les conduites à risques
   URL: https://www.paris.fr/pages/appel-a-projets-prevenir-les-cond...
   PDFs trouvés: 2
   
   1️⃣ aap_2026-2027_demarche-foyers-mmpcr-2-kzeJ.pdf
      URL: https://cdn.paris.fr/paris/2025/12/16/aap_2026-2027_demarche-foyers-mm...
      Co

## 8. Initialiser l'enrichisseur LLM

In [14]:
try:
    enricher = LLMEnricher()
    print("✅ LLMEnricher initialisé")
    print(f"   Modèle: claude-sonnet-4-5")
    print(f"   Retry logic: Enabled (max 3 attempts with exponential backoff)")
except ValueError as e:
    print(f"❌ {str(e)}")
    enricher = None

✅ LLMEnricher initialisé
   Modèle: claude-sonnet-4-5
   Retry logic: Enabled (max 3 attempts with exponential backoff)


## 9. Enrichir avec LLM

In [15]:
if enricher and mapped_df_paris is not None and len(mapped_df_paris) > 0:
    print(f"🔄 Enrichissement de {len(mapped_df_paris)} appels à projets...\n")
    
    for idx, row in mapped_df_paris.iterrows():
        url = row.get('url_source')
        titre = str(row.get('titre', 'N/A'))[:50]
        
        if not url or pd.isna(url):
            print(f"⏭️  [{idx+1}/{len(mapped_df_paris)}] {titre}: pas d'URL")
            continue
        
        print(f"🔄 [{idx+1}/{len(mapped_df_paris)}] {titre}...", end=' ')
        
        try:
            response = requests.get(url, headers=HEADERS, timeout=10)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.text, 'html.parser')
            pdf_links = find_pdf_links(soup, url)
            
            pdf_texts = []
            if pdf_links:
                print(f"(+{len(pdf_links)} PDFs)", end=' ')
                for pdf_url in pdf_links:
                    pdf_text = extract_pdf_text(pdf_url)
                    if pdf_text:
                        pdf_texts.append(pdf_text[:3000])
            
            extracted = enricher.extract_full_page(url, response.text, pdf_texts)
            
            if extracted:
                for key, value in extracted.items():
                    if key in mapped_df_paris.columns:
                        mapped_df_paris.at[idx, key] = value
                print("✅")
            else:
                print("⚠️  Aucune donnée")
                
        except requests.exceptions.Timeout:
            print("⏱️  Timeout")
        except Exception as e:
            print(f"❌ {str(e)[:30]}")
        
        time.sleep(1)
    
    print(f"\n✅ Enrichissement terminé!")
else:
    if enricher is None:
        print("❌ Enrichisseur LLM non disponible")
    if mapped_df_paris is None:
        print("❌ Pas de données")

🔄 Enrichissement de 12 appels à projets...

🔄 [1/12] Les appels à projets de la Ville de Paris... ✅
🔄 [2/12] Appel à projets pour le partage de la culture scie... (+2 PDFs) ✅
🔄 [3/12] Appel à projets « Prévenir les conduites à risques... (+2 PDFs) ✅
🔄 [4/12] Appel à projets « Paris Fabrik » : formations prof... (+2 PDFs) ✅
🔄 [5/12] Appel à projets immobiliers 2026 pour des lieux de... (+1 PDFs) ✅
🔄 [6/12] Appel à propositions pour un emplacement commercia... (+2 PDFs) ✅
🔄 [7/12] Appel à projets « Parcours linguistiques à visée p... (+2 PDFs) 

invalid pdf header: b'<!DOC'
EOF marker not found


  ⚠️ Erreur PDF https://www.paris.fr/pages/appel-a-projets-parcours-linguistiques-a-visee-professionnelle-plvp-2020-7465#reglement-documents-de-candidatures-et-faq: Stream has ended unexpectedly
✅
🔄 [8/12] Appel à projets : Actions de prévention de la pert... (+2 PDFs) ✅
🔄 [9/12] Appel à projets des activités péri et extrascolair... (+2 PDFs) ✅
🔄 [10/12] Appel à projets des temps d'activités périscolaire... (+2 PDFs) ✅
🔄 [11/12] N/A... ✅
🔄 [12/12] N/A... ✅

✅ Enrichissement terminé!


## 10. Statistiques et aperçu

In [16]:
if mapped_df_paris is not None:
    print("📊 Statistiques:")
    print(f"\n✅ Total: {len(mapped_df_paris)} enregistrements")
    print(f"\n✅ Remplissage:")
    for col in ['titre', 'resume', 'montant_max', 'montant_min', 'categories', 'public_cible', 'mots_cles', 'objectif']:
        if col in mapped_df_paris.columns:
            filled = mapped_df_paris[col].notna().sum()
            pct = (filled / len(mapped_df_paris) * 100) if len(mapped_df_paris) > 0 else 0
            print(f"   - {col}: {filled}/{len(mapped_df_paris)} ({pct:.1f}%)")

📊 Statistiques:

✅ Total: 12 enregistrements

✅ Remplissage:
   - titre: 12/12 (100.0%)
   - resume: 12/12 (100.0%)
   - montant_max: 1/12 (8.3%)
   - montant_min: 1/12 (8.3%)
   - categories: 12/12 (100.0%)
   - public_cible: 12/12 (100.0%)
   - mots_cles: 12/12 (100.0%)
   - objectif: 12/12 (100.0%)


In [17]:
if mapped_df_paris is not None and len(mapped_df_paris) > 0:
    print("\n📋 Aperçu:")
    print("\n" + "="*80)
    for idx in range(min(2, len(mapped_df_paris))):
        row = mapped_df_paris.iloc[idx]
        print(f"\n📌 {row['titre'][:60]}")
        print(f"   Org: {row['organisme']}")
        print(f"   Montant: {row['montant_max']}€")
        if pd.notna(row['resume']):
            print(f"   Résumé: {str(row['resume'])[:100]}...")


📋 Aperçu:


📌 Les appels à projets de la Ville de Paris
   Org: Mairie de Paris
   Montant: None€
   Résumé: La Ville de Paris propose divers appels à projets pour associations et structures : TAP, résidences ...

📌 Appel à projets pour le partage de la culture scientifique 2
   Org: Mairie de Paris
   Montant: 19000€
   Résumé: Appel à projets pour le partage de la culture scientifique à Paris en 2026. Favorise la diffusion de...


In [19]:
# Vue tabulaire simple
mapped_df_paris.to_string()

# Vue HTML interactive (meilleure pour Jupyter)
from IPython.display import HTML
display(HTML(mapped_df_paris.to_html()))

# Vue transposée (une ligne = une colonne)
mapped_df_paris.T

,titre,url_source,resume,date_publication,date_limite,montant_max,organisme,categories,id_record,public_cible,taux_financement,contact,modalite,demarches,mots_cles,objectif,montant_min,note,tags,perimetre_geo,fingerprint
0,Les appels à projets de la Ville de Paris,https://www.paris.fr/pages/repondre-a-un-appel-a-projets-5412,"La Ville de Paris propose divers appels à projets pour associations et structures : TAP, résidences artistiques, ESS, prévention, animation locale, culture scientifique, formations écologiques et animation de kiosques.",None,None,None,Mairie de Paris,"[Éducation, Culture, Jeunesse, Solidarité, Économie sociale et solidaire, Prévention, Animation locale, Formation professionnelle, Transition écologique, Culture scientifique]",paris_20260102203048_4825,"[associations, établissements publics, fondations, SIAE, ESUS, coopératives]",None,None,Dépôt de candidature via la plateforme Paris Asso dans les délais impartis pour chaque appel à projets. Les projets doivent s'inscrire dans le cadre défini par l'appel. Instruction technique puis sélection selon les critères du règlement.,"Enregistrer votre association sur Paris Asso, consulter les appels à projets en cours, déposer votre candidature en ligne avant la date limite spécifique à chaque appel","[appel à projets, Paris, associations, TAP, périscolaire, résidences artistiques, Art pour grandir, ESS, trophées, prévention rixes, fonds animation locale, parcours linguistiques, culture scientifique, conduites à risques, Paris Fabrik, transition écologique, kiosques, jeunesse]","Soutenir financièrement des projets associatifs et d'utilité sociale dans divers domaines (éducation, culture, jeunesse, solidarité, ESS, prévention) répondant aux problématiques définies par la Ville de Paris",None,None,None,Paris,bf30bdb77246
1,Appel à projets pour le partage de la culture scientifique 2026 à Paris,https://www.paris.fr/pages/appel-a-projets-2024-pour-le-partage-de-la-culture-scientifique-a-paris-25958,Appel à projets pour le partage de la culture scientifique à Paris en 2026. Favorise la diffusion de la culture scientifique auprès du grand public parisien.,None,2026-02-11,19000,Mairie de Paris,"[Culture, Sciences, Recherche, Education, Médiation scientifique]",paris_20260102203048_4099,"[associations, coopératives, fondations, organismes publics, organismes de recherche, universités]",Cofinancement obligatoire,None,"Projets devant démarrer avant fin décembre 2026, se dérouler à Paris, s'adresser au grand public. Cofinancement obligatoire. Budget total de 150 000 € réparti entre les lauréats. Annexe AAPCS 2026 obligatoire.",Dépôt dématérialisé via le service Subventions du portail Paris Asso avant le mercredi 11 février 2026 à 23h59. Une seule candidature par structure. Joindre l'annexe AAPCS 2026 et un compte-rendu financier si financé en 2025.,"[culture scientifique, recherche, grand public, Paris Recherche, médiation scientifique, jeunes publics, pluridisciplinarité, sciences humaines et sociales]",Favoriser le partage de la culture scientifique à travers le financement de projets mis en œuvre en 2026 sur le territoire parisien dans le cadre de la stratégie Paris Recherche,1000,None,None,Paris,305e6ffab869
2,Appel à projets « Prévenir les conduites à risques dans les établissements accueillant des adolescents et jeunes majeurs en situation de placement »,https://www.paris.fr/pages/appel-a-projets-prevenir-les-conduites-a-risques-dans-les-etablissements-accueillant-des-adolescents-et-jeunes-majeurs-en-situation-de-placement-33349,"Appel à projets pour renforcer la prévention des conduites à risques (addictions, numérique, vie affective) auprès des jeunes en situation de placement et des équipes qui les accompagnent.",2025-12-16,2026-02-16,None,Mairie de Paris,"[Santé publique, Prévention, Protection de l'enfance, Action sociale]",paris_20260102203048_1575,"[associations, structures de prévention, établissements d'accueil, structures intervenant auprès des 13-21 ans]",None

,0,1,2,3,4,5,6,7,8,9,10,11
titre,Les appels à projets de la Ville de Paris,Appel à projets pour le partage de la culture ...,Appel à projets « Prévenir les conduites à ris...,Appel à projets « Paris Fabrik » : formations ...,Appel à projets immobiliers 2026 pour des lieu...,Appel à propositions pour un emplacement comme...,Appel à projets « Parcours linguistiques à vis...,Appel à projets : Actions de prévention de la ...,Appel à projets des activités péri et extrasco...,Appel à projets des temps d'activités périscol...,N/A,N/A
url_source,https://www.paris.fr/pages/repondre-a-un-appel...,https://www.paris.fr/pages/appel-a-projets-202...,https://www.paris.fr/pages/appel-a-projets-pre...,https://www.paris.fr/pages/appel-a-projets-par...,https://www.paris.fr/pages/appel-a-projets-imm...,https://www.paris.fr/pages/appel-a-proposition...,https://www.paris.fr/pages/appel-a-projets-par...,https://www.paris.fr/pages/appel-a-projets-act...,https://www.paris.fr/pages/appel-a-projets-des...,https://www.paris.fr/pages/appel-a-projets-des...,https://www.paris.fr/appels-a-projets?page=2,https://www.paris.fr/appels-a-projets?page=2
resume,La Ville de Paris propose divers appels à proj...,Appel à projets pour le partage de la culture ...,Appel à projets pour renforcer la prévention d...,Appel à projets pour formations aux métiers de...,Appel à projets pour aider les opérateurs immo...,Appel à propositions pour un emplacement comme...,Appel à projets pour des formations linguistiq...,Appel à projets pour soutenir des actions de p...,Appel à projets pour des ateliers pédagogiques...,Appel à projets pour l'organisation d'ateliers...,Page regroupant les appels à projets de la Vil...,Page regroupant les différents appels à projet...
date_publication,None,None,2025-12-16,2025-12-15,2026-12-10,2025-12-05,2025-12-04,2025-11-21,2025-11-03,2025-11-03,None,None
date_limite,None,2026-02-11,2026-02-16,2026-02-19,2026-04-02,2026-01-14,2026-02-03,2026-01-09,2026-01-05,2026-01-05,None,None
montant_max,None,19000,None,None,None,None,None,None,None,None,None,None
organisme,Mairie de Paris,Mairie de Paris,Mairie de Paris,Mairie de Paris,Mairie de Paris,Mairie de Paris,Mairie de Paris,Mairie de Paris,Mairie de Paris,Mairie de Paris,Mairie de Paris,Mairie de Paris
categories,"[Éducation, Culture, Jeunesse, Solidarité, Éco...","[Culture, Sciences, Recherche, Education, Médi...","[Santé publique, Prévention, Protection de l'e...","[Formation professionnelle, Transition écologi...","[Economie sociale et solidaire, Economie circu...","[Commerce, Innovation, Numérique, Occupation d...","[Emploi, Formation, Insertion professionnelle,...","[Social, Santé, Prévention, Seniors, Autonomie]","[Éducation, Urbanisme, Périscolaire, Renouvell...","[Éducation, Périscolaire, Enfance, Animation, ...","[Recherche, Jeunesse, Culture, Vie associative...","[Appels à projets, Culture, Jeunesse, Vacances..."
id_record,paris_20260102203048_4825,paris_20260102203048_4099,paris_20260102203048_1575,paris_20260102203048_4708,paris_20260102203048_5197,paris_20260102203048_9238,paris_20260102203048_6556,paris_20260102203048_2858,paris_20260102203048_6092,paris_20260102203048_8528,paris_20260102203048_942,paris_20260102203048_942
public_cible,"[associations, établissements publics, fondati...","[associations, coopératives, fondations, organ...","[associations, structures de prévention, établ...","[Organismes de formation, Associations, Struct...","[Bailleurs, Foncières, Promoteurs, Opérateurs ...","[Entreprises, Start-ups, Professionnels]","[Associations, Fondations, Établissements publ...","[associations, fondations, gestionnaires de ré...","[associations, organismes éducatifs, établisse...","[associations, organismes proposant des atelie...","[Associations, Professionnels, Artistes, Collé...","[Professionnels, Associations, Artistes, Collé..."


## 11. Exporter les données

In [ ]:
if mapped_df_paris is not None:
    # Exporter en CSV
    csv_output = '../data/paris_aap_enriched.csv'
    try:
        os.makedirs(os.path.dirname(csv_output), exist_ok=True)
        mapped_df_paris.to_csv(csv_output, index=False, encoding='utf-8')
        print(f"✅ Exporté en CSV: {csv_output}")
    except Exception as e:
        print(f"⚠️  Erreur CSV: {str(e)}")
    
    # Exporter en JSON
    json_output = '../data/paris_aap_enriched.json'
    try:
        os.makedirs(os.path.dirname(json_output), exist_ok=True)
        df_for_json = mapped_df_paris.copy()
        for col in df_for_json.columns:
            if df_for_json[col].dtype == 'object':
                df_for_json[col] = df_for_json[col].astype(str)
        
        df_for_json.to_json(json_output, orient='records', force_ascii=False, indent=2)
        print(f"✅ Exporté en JSON: {json_output}")
    except Exception as e:
        print(f"⚠️  Erreur JSON: {str(e)}")

## 12. Upload Airtable (optionnel)

In [ ]:
if mapped_df_paris is not None:
    # Décommentez pour uploader
    # from appels_a_projets.connectors.airtable_connector import AirtableConnector
    # 
    # airtable = AirtableConnector()
    # uploaded = airtable.upload_dataframe(mapped_df_paris)
    # print(f"✅ Uploaded {uploaded} records to Airtable!")
    
    print("⏸️  Upload Airtable optionnel (décommenter si besoin)")